In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
from scipy.stats import norm
import scipy.integrate as integrate

import gym
from gym import spaces

import random
import itertools as it
from joblib import Parallel, delayed
from toolz import memoize
from contracts import contract
from collections import namedtuple, defaultdict, deque, Counter

import warnings
warnings.filterwarnings("ignore", 
                        message="The objective has been evaluated at this point before.")

from agents import Agent
from oldmouselab import OldMouselabEnv
from policies import FixedPlanPolicy, LiederPolicy
from evaluation import *
from distributions import cmax, smax, sample, expectation, Normal, PointMass, SampleDist, Normal, Categorical

In [3]:
def hd_dist(attributes):
    dist = [1,]*attributes
    dist[0] = np.random.randint(85,97)
    for i in range(1,attributes-1):
        dist[i] += np.random.randint(0,100-np.sum(dist))
    dist[-1] += 100-np.sum(dist)
    dist = np.around(np.array(dist)/100,decimals=2)
    np.random.shuffle(dist)
    return dist

def ld_dist(attributes):
    dist = [np.random.randint(10,40) for _ in range(attributes)]
    dist = np.around(np.array(dist)/sum(dist),decimals=2)
    np.random.shuffle(dist)
    return dist

In [4]:
gambles = 7
attributes = 4
high_stakes = Normal((9.99+0.01)/2, 0.3*(9.99-0.01))
low_stakes = Normal((0.25+0.01)/2, 0.3*(0.25-0.01))
reward = high_stakes
cost=.03

#set to 20 for sanity check
n_train = 20
n_test = 20

train_envs_hd = [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs = train_envs_hd+train_envs_ld 

test_envs_hd =  [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs = test_envs_hd+test_envs_ld 

term_action = train_envs[0].term_action

In [5]:
bo_pol_theta = np.load('data/high_stakes_3cents.npy')
bo_pol = LiederPolicy(list(bo_pol_theta))

In [42]:
agent = Agent()
def run_env(policy, env):
    agent.register(env)
    agent.register(policy)
    tr = agent.run_episode()
#     print(tr)
    return {'util': tr['return'], 'actions': tr['actions'],
            'observations': len(tr['actions']) - 1, 'ground_truth': env.ground_truth}

def action_coordinate(env, action):
    return (action//env.outcomes,action%env.outcomes)

def p_grid(env, actions):
    grid = np.zeros((env.gambles+1,env.outcomes))
    grid[0,:] = env.dist
    for i in range(len(actions[:-1])):
        gamble, outcome = action_coordinate(env,actions[i]) 
        grid[gamble+1, outcome] = i+1
    return grid

In [36]:
train_envs[21].reset()
trace = run_env(bo_pol, train_envs[21])
trace

{'actions': [15, 11, 3, 7, 23, 19, 18, 14, 27, 16, 12, 6, 4, 20, 2, 28],
 'ground_truth': array([  5.72 ,   3.217,   2.329,   4.651,   6.133,   5.194,   9.519,   5.233,   6.989,  -2.191,  10.047,   2.322,   1.857,  10.419,   2.03 ,   8.164,   3.851,   5.734,   6.694,   6.005,   2.839,
         10.503,   3.574,   4.335,   5.184,  10.58 ,   7.042,   5.057]),
 'observations': 15,
 'util': 6.0173855517845771}

In [37]:
train_envs[21].dist

array([ 0.23,  0.19,  0.25,  0.33])

In [38]:
train_envs[21].grid()

array([[Norm(5.00, 2.99), Norm(5.00, 2.99), 2.3291105682994981, 4.6506266465148158],
       [6.1326595288813932, Norm(5.00, 2.99), 9.5193876241208599, 5.2334150124595205],
       [Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99), 2.3220610718772274],
       [1.8569011494096928, Norm(5.00, 2.99), 2.0300837314080731, 8.1643066308078609],
       [3.8510808099828533, Norm(5.00, 2.99), 6.6942617780405316, 6.0054604865583121],
       [2.8387405059890871, Norm(5.00, 2.99), Norm(5.00, 2.99), 4.335424968102747],
       [Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99), 5.0567247344048454]], dtype=object)

In [43]:
p_grid(train_envs[21],trace['actions'])

array([[  0.23,   0.19,   0.25,   0.33],
       [  0.  ,   0.  ,  15.  ,   3.  ],
       [ 13.  ,   0.  ,  12.  ,   4.  ],
       [  0.  ,   0.  ,   0.  ,   2.  ],
       [ 11.  ,   0.  ,   8.  ,   1.  ],
       [ 10.  ,   0.  ,   7.  ,   6.  ],
       [ 14.  ,   0.  ,   0.  ,   5.  ],
       [  0.  ,   0.  ,   0.  ,   9.  ]])